<h2>Part 1. Building the table from wikipedia page</h2>

In [11]:
# importing libraries needed 
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import urllib.request
from bs4 import BeautifulSoup


In [12]:
# parsing wiki page into html 

fp = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
mybytes = fp.read()

mystr = mybytes.decode("utf8")
fp.close()

In [13]:
# parsing html into BS soup object

soup = BeautifulSoup(mystr, 'html.parser')


In [14]:
# creting lists for each of the parsed table rows

table = soup.find('table', attrs={'class':'wikitable'})
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.replace('\n','') for tr in td]
    l.append(row)
l[0:10]

[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village']]

In [15]:
# creting header columns for each of the table headers

col_rows=table.find_all('th')

cols=[]
for th in col_rows:
    row=th.text.replace('\n','')
    cols.append(row)
cols

['Postal Code', 'Borough', 'Neighborhood']

In [16]:
# building a dataframe

df=pd.DataFrame(l, columns=cols)
df = df.iloc[1:]
df.head()

Postal Code           Borough               Neighborhood
1         M1A      Not assigned               Not assigned
2         M2A      Not assigned               Not assigned
3         M3A        North York                  Parkwoods
4         M4A        North York           Victoria Village
5         M5A  Downtown Toronto  Regent Park, Harbourfront

In [17]:
# dropping the boroughts which are not assigned

df=df[df['Borough']!='Not assigned']

In [18]:
# checking if there are duplicated postal codes for neighbors. however, no duplicates and we can work with DF dataframe

dfs=df

dfs['Neighborhood'] = dfs[['Postal Code','Borough','Neighborhood']].groupby(['Postal Code','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
dfs=dfs.drop_duplicates()

In [20]:
df.head()

Postal Code           Borough                                 Neighborhood
3         M3A        North York                                    Parkwoods
4         M4A        North York                             Victoria Village
5         M5A  Downtown Toronto                    Regent Park, Harbourfront
6         M6A        North York             Lawrence Manor, Lawrence Heights
7         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [19]:
df.shape

(103, 3)